<a href="https://colab.research.google.com/github/Meet2147/Advanced-Computer-Vision/blob/master/WGAN_GP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import torch
import torchvision
import numpy as np
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

# Image processing
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.5,),
                                     std=(0.5,))])

# MNIST dataset
mnist = torchvision.datasets.MNIST(root='../../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1))

# Generator 
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

# Device setting
D = D.to(device)
G = G.to(device)

# Optimizers
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))

def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()
    
    
# Loss weight for gradient penalty
lambda_gp = 10


def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = torch.Tensor(np.random.random((real_samples.size(0),1))).to(device)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = torch.autograd.Variable(torch.Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False).to(device)
    # Get gradient w.r.t. interpolates
    gradients = torch.autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty
  

# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #
        for di in range(5):
          z = torch.randn(batch_size, latent_size).to(device)
          fake_images = G(z)
          D_real = D(images)
          D_fake = D(fake_images)
          
          # Gradient penalty
          gradient_penalty = compute_gradient_penalty(D, images.data, fake_images.data)
        
          # Backprop and optimize
          d_loss = -(torch.mean(D_real) - torch.mean(D_fake)) + lambda_gp * gradient_penalty
          
          reset_grad()
          d_loss.backward()
          d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        D_fake = D(fake_images)
        
        g_loss = -torch.mean(D_fake)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          D_real.mean().item(), D_fake.mean().item()))
    
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

# Save the model checkpoints 
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')

Epoch [0/200], Step [200/600], d_loss: -7.8255, g_loss: -2.0063, D(x): 11.18, D(G(z)): 2.01
Epoch [0/200], Step [400/600], d_loss: -7.7546, g_loss: -4.0642, D(x): 13.16, D(G(z)): 4.06
Epoch [0/200], Step [600/600], d_loss: -5.9423, g_loss: -3.0656, D(x): 9.71, D(G(z)): 3.07
Epoch [1/200], Step [200/600], d_loss: -5.1979, g_loss: -3.7847, D(x): 9.32, D(G(z)): 3.78
Epoch [1/200], Step [400/600], d_loss: -4.9061, g_loss: 0.4708, D(x): 4.79, D(G(z)): -0.47
Epoch [1/200], Step [600/600], d_loss: -5.1692, g_loss: 2.8276, D(x): 2.81, D(G(z)): -2.83
Epoch [2/200], Step [200/600], d_loss: -4.5238, g_loss: 3.2357, D(x): 1.84, D(G(z)): -3.24
Epoch [2/200], Step [400/600], d_loss: -5.3324, g_loss: 2.4621, D(x): 3.27, D(G(z)): -2.46
Epoch [2/200], Step [600/600], d_loss: -5.2905, g_loss: 5.1131, D(x): 0.43, D(G(z)): -5.11
Epoch [3/200], Step [200/600], d_loss: -4.8454, g_loss: 3.5990, D(x): 1.88, D(G(z)): -3.60
Epoch [3/200], Step [400/600], d_loss: -5.1383, g_loss: 4.0809, D(x): 1.43, D(G(z)): -4.